In [1]:
import pandas as pd 
from glob import glob #檔名模式匹配(filename pattern matching)模組(Module)
import numpy as np 
import nltk #英語NLP套件
import re
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *

### 載入爬蟲文本資料 ->髒亂 還有 缺失。

In [3]:
#獲取所有AUD的新聞路徑
files = glob('./Newsdata/CurrencyNews_1*/AUD*.csv')
print(files)

['./Newsdata\\CurrencyNews_105\\AUD10501.csv', './Newsdata\\CurrencyNews_105\\AUD10502.csv', './Newsdata\\CurrencyNews_105\\AUD10503.csv', './Newsdata\\CurrencyNews_105\\AUD10504.csv', './Newsdata\\CurrencyNews_105\\AUD10505.csv', './Newsdata\\CurrencyNews_105\\AUD10506.csv', './Newsdata\\CurrencyNews_105\\AUD10507.csv', './Newsdata\\CurrencyNews_105\\AUD10508.csv', './Newsdata\\CurrencyNews_105\\AUD10509.csv', './Newsdata\\CurrencyNews_105\\AUD10510.csv', './Newsdata\\CurrencyNews_105\\AUD10511.csv', './Newsdata\\CurrencyNews_105\\AUD10512.csv', './Newsdata\\CurrencyNews_106\\AUD10601.csv', './Newsdata\\CurrencyNews_106\\AUD10602.csv', './Newsdata\\CurrencyNews_106\\AUD10603.csv', './Newsdata\\CurrencyNews_106\\AUD10604.csv', './Newsdata\\CurrencyNews_106\\AUD10605.csv', './Newsdata\\CurrencyNews_106\\AUD10606.csv', './Newsdata\\CurrencyNews_106\\AUD10607.csv', './Newsdata\\CurrencyNews_106\\AUD10608.csv', './Newsdata\\CurrencyNews_106\\AUD10609.csv', './Newsdata\\CurrencyNews_106\\AU

In [5]:
##載入資料(並選擇新聞本文欄位)

data = pd.DataFrame()

for file in files:
    #只要'date','body','source_title'這幾個欄位，編碼為ANSI
    df = pd.read_csv(file, usecols=['date','body','source_title'],
                    dtype={ 'date': str, 'body':str,'source_title':str},
                    encoding = "ANSI")
    #反轉index順序以符合時間
    df = df.reindex(index=df.index[::-1])
    #把所有df合併
    data = pd.concat([data,df],ignore_index=True)
#重設index
df_news = data.reset_index(drop=True)
df_news

,date,body,source_title
0,2016-01-01,Roger Federer is a man used to making history....,Forbes
1,2016-01-03,We're continually told that raising the minimu...,Forbes
2,2016-01-04,"If you have a large tax debt, you might have t...",Forbes
3,2016-01-04,"As markets reopen for the first week of 2016, ...",CNBC
4,2016-01-04,Good morning. Here's everything you need to kn...,Business Insider
...,...,...,...
17475,2020-12-31,"The Los Olivos, Calif., property underwent a s...",The Wall Street Journal
17476,2020-12-31,A look at the biggest residential transactions...,The Wall Street Journal
17477,2020-12-31,I write about global policy and lead advocacy ...,Forbes
17478,2020-12-31,Buoyed by positive momentum for shares of Inte...,MarketWatch


In [6]:
# 刪除遺漏值
df_news = df_news.dropna()
df_news = df_news.reset_index(drop = True)

In [7]:
df_news.shape

(17480, 3)

### 處理時間

In [8]:
#確認時間是否正確 ->2021/5 的日期格式不對。
for i in df_news["date"]:
    print(i)

2016-01-01
2016-01-03
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-07
2016-01-08
2016-01-08
2016-01-08
2016-01-08
2016-01-08
2016-01-10
2016-01-10
2016-01-10
2016-01-10
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-14
2016-01-14
2016-01-14
2016-01-14
2016-01-15
2016-01-15
2016-01-15
2016-01-15
2016-01-15
2016-01-16
2016-01-17
2016-01-17
2016-01-17
2016-01-17
2016-01-18
2016-01-18
2016-01-18
2016-01-18
2016-01-19
2016-01-19
2016-01-19
2016-01-19
2016-01-19
2016-01-19
2016-01-19
2016-01-20
2016-01-20
2016-01-20
2016-01-20
2016-01-21
2016-01-21
2016-01-21
2016-01-21
2016-01-21
2016-01-22
2016-01-22

2016-11-01
2016-11-01
2016-11-01
2016-11-02
2016-11-02
2016-11-02
2016-11-02
2016-11-02
2016-11-02
2016-11-02
2016-11-02
2016-11-02
2016-11-02
2016-11-02
2016-11-03
2016-11-03
2016-11-03
2016-11-03
2016-11-03
2016-11-03
2016-11-03
2016-11-03
2016-11-03
2016-11-03
2016-11-04
2016-11-04
2016-11-04
2016-11-04
2016-11-05
2016-11-06
2016-11-06
2016-11-06
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-07
2016-11-08
2016-11-08
2016-11-08
2016-11-08
2016-11-08
2016-11-08
2016-11-08
2016-11-09
2016-11-09
2016-11-09
2016-11-09
2016-11-10
2016-11-10
2016-11-10
2016-11-10
2016-11-10
2016-11-11
2016-11-11
2016-11-11
2016-11-11
2016-11-11
2016-11-11
2016-11-12
2016-11-12
2016-11-12
2016-11-12
2016-11-13
2016-11-13
2016-11-14
2016-11-14
2016-11-14
2016-11-14
2016-11-14
2016-11-15
2016-11-15
2016-11-15
2016-11-16
2016-11-16
2016-11-16
2016-11-16
2016-11-16
2016-11-16
2016-11-16
2016-11-17

2017-09-29
2017-09-29
2017-09-30
2017-09-30
2017-09-30
2017-10-01
2017-10-01
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-03
2017-10-03
2017-10-03
2017-10-03
2017-10-03
2017-10-04
2017-10-04
2017-10-04
2017-10-04
2017-10-04
2017-10-04
2017-10-04
2017-10-05
2017-10-05
2017-10-05
2017-10-05
2017-10-05
2017-10-05
2017-10-05
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-07
2017-10-07
2017-10-08
2017-10-08
2017-10-09
2017-10-09
2017-10-09
2017-10-09
2017-10-09
2017-10-09
2017-10-09
2017-10-09
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-10
2017-10-11
2017-10-11
2017-10-11
2017-10-11
2017-10-12
2017-10-12
2017-10-12
2017-10-12
2017-10-13
2017-10-13
2017-10-13
2017-10-13
2017-10-13
2017-10-13
2017-10-15
2017-10-16
2017-10-16
2017-10-16
2017-10-16
2017-10-17
2017-10-17
2017-10-17

2018-06-05
2018-06-05
2018-06-05
2018-06-05
2018-06-05
2018-06-05
2018-06-05
2018-06-05
2018-06-05
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-06
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-07
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-08
2018-06-09
2018-06-09
2018-06-09
2018-06-09
2018-06-09
2018-06-09
2018-06-09
2018-06-09
2018-06-09
2018-06-09
2018-06-10
2018-06-10
2018-06-10
2018-06-11
2018-06-11
2018-06-11
2018-06-11
2018-06-11
2018-06-12
2018-06-12
2018-06-12
2018-06-12
2018-06-12
2018-06-12
2018-06-12
2018-06-12
2018-06-12
2018-06-12
2018-06-12
2018-06-12
2018-06-13

2018-11-28
2018-11-28
2018-11-28
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-29
2018-11-30
2018-11-30
2018-11-30
2018-11-30
2018-11-30
2018-11-30
2018-11-30
2018-11-30
2018-11-30
2018-11-30
2018-12-01
2018-12-01
2018-12-01
2018-12-02
2018-12-02
2018-12-02
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-03
2018-12-04
2018-12-04
2018-12-04
2018-12-04
2018-12-04
2018-12-05
2018-12-05
2018-12-05
2018-12-05
2018-12-05
2018-12-05
2018-12-05
2018-12-05
2018-12-05
2018-12-05
2018-12-05
2018-12-05
2018-12-06
2018-12-06
2018-12-06
2018-12-06
2018-12-06
2018-12-06
2018-12-06
2018-12-06
2018-12-06
2018-12-06
2018-12-06
2018-12-06
2018-12-07
2018-12-07
2018-12-07
2018-12-07

2019-07-27
2019-07-28
2019-07-28
2019-07-28
2019-07-28
2019-07-28
2019-07-28
2019-07-29
2019-07-29
2019-07-29
2019-07-29
2019-07-29
2019-07-29
2019-07-29
2019-07-29
2019-07-29
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-31
2019-07-31
2019-07-31
2019-07-31
2019-07-31
2019-07-31
2019-07-31
2019-07-31
2019-07-31
2019-07-31
2019-07-31
2019-07-31
2019-08-01
2019-08-01
2019-08-01
2019-08-01
2019-08-01
2019-08-01
2019-08-01
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-02
2019-08-03
2019-08-03
2019-08-03
2019-08-03
2019-08-04
2019-08-04
2019-08-04
2019-08-05
2019-08-05
2019-08-05
2019-08-05
2019-08-05
2019-08-05
2019-08-05
2019-08-05
2019-08-05
2019-08-05
2019-08-05
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06
2019-08-06

2019-12-15
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-16
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-17
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-18
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-19
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20
2019-12-20

2020-07-01
2020-07-01
2020-07-01
2020-07-01
2020-07-02
2020-07-02
2020-07-02
2020-07-02
2020-07-02
2020-07-02
2020-07-02
2020-07-02
2020-07-02
2020-07-02
2020-07-03
2020-07-03
2020-07-03
2020-07-03
2020-07-03
2020-07-03
2020-07-05
2020-07-05
2020-07-05
2020-07-05
2020-07-05
2020-07-05
2020-07-05
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-06
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-07
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-08
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09
2020-07-09

2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-14
2020-11-14
2020-11-14
2020-11-15
2020-11-15
2020-11-15
2020-11-15
2020-11-16
2020-11-16
2020-11-16
2020-11-16
2020-11-16
2020-11-16
2020-11-16
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-19
2020-11-19
2020-11-19
2020-11-19
2020-11-19
2020-11-19
2020-11-19
2020-11-19
2020-11-19
2020-11-20
2020-11-20
2020-11-20
2020-11-20
2020-11-20
2020-11-20
2020-11-20
2020-11-20
2020-11-20
2020-11-20
2020-11-20
2020-11-20
2020-11-21
2020-11-21
2020-11-21
2020-11-21
2020-11-22
2020-11-22
2020-11-22
2020-11-23
2020-11-23
2020-11-23
2020-11-23
2020-11-23
2020-11-23
2020-11-23
2020-11-23
2020-11-23
2020-11-23
2020-11-23

In [29]:
# 因為沒有2021的資料了，所以下列不用做
print(df_news[df_news["date"] == "2021/5/1"].index)

Int64Index([105054, 105055, 105056, 105057, 105058, 105059, 105060, 105061,
            105062, 105063, 105064, 105065, 105066],
           dtype='int64')


In [32]:
for i in range(df_news.shape[0]-105054):
    df_news["date"][i+105054] = str(df_news["date"][i+105054]).replace("/5/5","-05-05")
    df_news["date"][i+105054] = str(df_news["date"][i+105054]).replace("/5/4","-05-04")
    df_news["date"][i+105054] = str(df_news["date"][i+105054]).replace("/5/6","-05-06")
    df_news["date"][i+105054] = str(df_news["date"][i+105054]).replace("/5/7","-05-07")
    df_news["date"][i+105054] = str(df_news["date"][i+105054]).replace("/5/8","-05-08")
    df_news["date"][i+105054] = str(df_news["date"][i+105054]).replace("/5/9","-05-09")
    df_news["date"][i+105054] = str(df_news["date"][i+105054]).replace("/5","-05")
    df_news["date"][i+105054] = str(df_news["date"][i+105054]).replace("/","-")

In [33]:
for i in range(df_news.shape[0]-105054):
    if df_news["date"][i+105054] == "2021-05-1":
        df_news["date"][i+105054] = "2021-05-01"
    elif df_news["date"][i+105054] == "2021-05-2":
        df_news["date"][i+105054] = "2021-05-02"
    elif df_news["date"][i+105054] == "2021-05-3":
        df_news["date"][i+105054] = "2021-05-03"

In [34]:
for i in df_news["date"]:
    print(i)

2016-01-01
2016-01-01
2016-01-01
2016-01-01
2016-01-01
2016-01-01
2016-01-01
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06

2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-25
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26
2016-02-26

2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-06
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07
2016-04-07

2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-17
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18
2016-05-18

2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-20
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21
2016-06-21

...

2020-09-22
2020-09-22
2020-09-22
2020-09-22
2020-09-22
2020-09-22
2020-09-22
2020-09-22
2020-09-22
2020-09-22
2020-09-22
2020-09-22
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-23
2020-09-24
2020-09-24
2020-09-24
2020-09-24
2020-09-24
2020-09-24
2020-09-24
2020-09-24
2020-09-24
2020-09-24
2020-09-24

2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-12
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-13
2020-11-14
2020-11-14

2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-11
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12
2021-01-12

2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-05
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-06
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-07
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08
2021-03-08

2021-04-27
2021-04-27
2021-04-27
2021-04-27
2021-04-27
2021-04-27
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28
2021-04-28

### 把奇怪的日期處理掉

In [9]:
df_news = df_news[df_news["date"].str.contains("[0-9]*-[0-9]*-[09]*")]

In [10]:
for i in df_news["date"]:
    print(i)

2016-01-01
2016-01-03
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-07
2016-01-08
2016-01-08
2016-01-08
2016-01-08
2016-01-08
2016-01-10
2016-01-10
2016-01-10
2016-01-10
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-11
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-12
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-13
2016-01-14
2016-01-14
2016-01-14
2016-01-14
2016-01-15
2016-01-15
2016-01-15
2016-01-15
2016-01-15
2016-01-16
2016-01-17
2016-01-17
2016-01-17
2016-01-17
2016-01-18
2016-01-18
2016-01-18
2016-01-18
2016-01-19
2016-01-19
2016-01-19
2016-01-19
2016-01-19
2016-01-19
2016-01-19
2016-01-20
2016-01-20
2016-01-20
2016-01-20
2016-01-21
2016-01-21
2016-01-21
2016-01-21
2016-01-21
2016-01-22
2016-01-22

2016-11-11
2016-11-11
2016-11-11
2016-11-11
2016-11-12
2016-11-12
2016-11-12
2016-11-12
2016-11-13
2016-11-13
2016-11-14
2016-11-14
2016-11-14
2016-11-14
2016-11-14
2016-11-15
2016-11-15
2016-11-15
2016-11-16
2016-11-16
2016-11-16
2016-11-16
2016-11-16
2016-11-16
2016-11-16
2016-11-17
2016-11-17
2016-11-17
2016-11-17
2016-11-18
2016-11-18
2016-11-18
2016-11-18
2016-11-18
2016-11-18
2016-11-19
2016-11-20
2016-11-20
2016-11-20
2016-11-21
2016-11-21
2016-11-21
2016-11-21
2016-11-21
2016-11-21
2016-11-21
2016-11-21
2016-11-21
2016-11-21
2016-11-21
2016-11-21
2016-11-22
2016-11-22
2016-11-22
2016-11-22
2016-11-22
2016-11-22
2016-11-22
2016-11-23
2016-11-23
2016-11-23
2016-11-24
2016-11-25
2016-11-25
2016-11-25
2016-11-25
2016-11-25
2016-11-27
2016-11-27
2016-11-27
2016-11-27
2016-11-28
2016-11-28
2016-11-28
2016-11-28
2016-11-28
2016-11-28
2016-11-28
2016-11-28
2016-11-28
2016-11-29
2016-11-29
2016-11-29
2016-11-29
2016-11-29
2016-11-29
2016-11-29
2016-11-29
2016-11-29
2016-11-29
2016-11-29

2017-09-20
2017-09-20
2017-09-21
2017-09-21
2017-09-21
2017-09-21
2017-09-21
2017-09-21
2017-09-22
2017-09-22
2017-09-22
2017-09-22
2017-09-22
2017-09-22
2017-09-23
2017-09-23
2017-09-24
2017-09-24
2017-09-24
2017-09-25
2017-09-25
2017-09-25
2017-09-25
2017-09-25
2017-09-26
2017-09-26
2017-09-26
2017-09-26
2017-09-28
2017-09-28
2017-09-28
2017-09-28
2017-09-28
2017-09-28
2017-09-28
2017-09-28
2017-09-29
2017-09-29
2017-09-29
2017-09-29
2017-09-29
2017-09-29
2017-09-30
2017-09-30
2017-09-30
2017-10-01
2017-10-01
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-02
2017-10-03
2017-10-03
2017-10-03
2017-10-03
2017-10-03
2017-10-04
2017-10-04
2017-10-04
2017-10-04
2017-10-04
2017-10-04
2017-10-04
2017-10-05
2017-10-05
2017-10-05
2017-10-05
2017-10-05
2017-10-05
2017-10-05
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-06
2017-10-07
2017-10-07
2017-10-08
2017-10-08
2017-10-09
2017-10-09
2017-10-09
2017-10-09
2017-10-09

2018-05-12
2018-05-12
2018-05-12
2018-05-13
2018-05-13
2018-05-13
2018-05-13
2018-05-13
2018-05-13
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-14
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-15
2018-05-16
2018-05-16
2018-05-16
2018-05-16
2018-05-16
2018-05-16
2018-05-16
2018-05-16
2018-05-16
2018-05-16
2018-05-16
2018-05-16
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-17
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-18
2018-05-19
2018-05-19
2018-05-20
2018-05-21
2018-05-21

2018-11-02
2018-11-02
2018-11-02
2018-11-02
2018-11-02
2018-11-03
2018-11-03
2018-11-04
2018-11-04
2018-11-04
2018-11-04
2018-11-04
2018-11-05
2018-11-05
2018-11-05
2018-11-05
2018-11-05
2018-11-05
2018-11-05
2018-11-05
2018-11-05
2018-11-06
2018-11-06
2018-11-06
2018-11-06
2018-11-06
2018-11-06
2018-11-06
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-07
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-08
2018-11-09
2018-11-09
2018-11-09
2018-11-09
2018-11-09
2018-11-09
2018-11-09
2018-11-10
2018-11-10
2018-11-10
2018-11-10
2018-11-10
2018-11-10
2018-11-10
2018-11-10
2018-11-10
2018-11-11
2018-11-11
2018-11-11
2018-11-12
2018-11-12
2018-11-12
2018-11-12
2018-11-12
2018-11-12
2018-11-12
2018-11-12
2018-11-12
2018-11-12
2018-11-12

2019-08-10
2019-08-10
2019-08-10
2019-08-10
2019-08-10
2019-08-10
2019-08-11
2019-08-11
2019-08-11
2019-08-11
2019-08-11
2019-08-12
2019-08-12
2019-08-12
2019-08-12
2019-08-12
2019-08-12
2019-08-12
2019-08-12
2019-08-12
2019-08-12
2019-08-12
2019-08-13
2019-08-13
2019-08-13
2019-08-13
2019-08-13
2019-08-13
2019-08-13
2019-08-13
2019-08-14
2019-08-14
2019-08-14
2019-08-14
2019-08-14
2019-08-14
2019-08-15
2019-08-15
2019-08-15
2019-08-15
2019-08-15
2019-08-15
2019-08-15
2019-08-16
2019-08-16
2019-08-16
2019-08-16
2019-08-16
2019-08-16
2019-08-16
2019-08-16
2019-08-17
2019-08-17
2019-08-18
2019-08-18
2019-08-18
2019-08-18
2019-08-19
2019-08-19
2019-08-19
2019-08-19
2019-08-20
2019-08-20
2019-08-20
2019-08-20
2019-08-20
2019-08-20
2019-08-20
2019-08-20
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21
2019-08-21

2019-11-01
2019-11-01
2019-11-01
2019-11-01
2019-11-01
2019-11-01
2019-11-01
2019-11-01
2019-11-02
2019-11-02
2019-11-02
2019-11-02
2019-11-02
2019-11-03
2019-11-03
2019-11-03
2019-11-03
2019-11-03
2019-11-03
2019-11-03
2019-11-03
2019-11-03
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-04
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-05
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-06
2019-11-07
2019-11-07
2019-11-07
2019-11-07

2020-05-12
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-13
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-14
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-15
2020-05-16
2020-05-16
2020-05-16
2020-05-16
2020-05-16
2020-05-16
2020-05-16
2020-05-17
2020-05-17
2020-05-17
2020-05-17
2020-05-17
2020-05-17
2020-05-17
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-18
2020-05-19
2020-05-19
2020-05-19
2020-05-19

2020-10-13
2020-10-13
2020-10-13
2020-10-13
2020-10-13
2020-10-13
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-14
2020-10-15
2020-10-15
2020-10-15
2020-10-15
2020-10-15
2020-10-15
2020-10-15
2020-10-15
2020-10-15
2020-10-15
2020-10-16
2020-10-16
2020-10-16
2020-10-16
2020-10-16
2020-10-16
2020-10-16
2020-10-16
2020-10-16
2020-10-16
2020-10-17
2020-10-17
2020-10-17
2020-10-17
2020-10-17
2020-10-17
2020-10-18
2020-10-18
2020-10-18
2020-10-18
2020-10-18
2020-10-18
2020-10-19
2020-10-19
2020-10-19
2020-10-19
2020-10-19
2020-10-19
2020-10-19
2020-10-19
2020-10-19
2020-10-19
2020-10-19
2020-10-19
2020-10-20
2020-10-20
2020-10-20
2020-10-20
2020-10-20
2020-10-20
2020-10-20
2020-10-20
2020-10-20
2020-10-21
2020-10-21
2020-10-21
2020-10-21
2020-10-21
2020-10-21
2020-10-21
2020-10-21
2020-10-21
2020-10-21
2020-10-21
2020-10-21
2020-10-22
2020-10-22

### 處理新聞title

In [11]:
print(df_news.shape[0])
#瞭解個來源新聞分布數量
print(df_news.drop_duplicates(subset = "source_title"))

17480
             date                                               body  \
0      2016-01-01  Roger Federer is a man used to making history....   
3      2016-01-04  As markets reopen for the first week of 2016, ...   
4      2016-01-04  Good morning. Here's everything you need to kn...   
5      2016-01-04  The global financial markets in 2015 were pull...   
6      2016-01-04  Exporters increase currency protection to lock...   
13     2016-01-05  LAS VEGAS -- For ad-supported tech companies -...   
15     2016-01-06  The yen climbed against its rival currencies i...   
16     2016-01-06  Delivering up-to-the minute news, analysis, in...   
35     2016-01-11  Japan's Soryu-class submarines have technical ...   
40     2016-01-12  Eaton earnings are likely to be able to increa...   
50     2016-01-13  SHANGHAI (Reuters) - China's central bank held...   
13249  2020-01-02  On a dusty airfield in a remote outback town, ...   

                       source_title  
0                  

In [12]:
dic = {} #資料有的tilte都加進去
news_resource = {} #只放有大於五篇以上的title
for i in df_news["source_title"]:
    if i not in dic:
        dic[i] = 1
    elif i in dic:
        dic[i] = dic[i]+1
        
for key, value in dic.items():
    if dic[key] > 5:
        news_resource[key] = value

In [13]:
#整理重複來源
news_resource

{'Forbes': 908,
 'CNBC': 1655,
 'Business Insider': 937,
 'Seeking Alpha': 3572,
 'Bloomberg Business': 2383,
 'International Business Times': 267,
 'MarketWatch': 1599,
 'The Wall Street Journal': 931,
 'Financial Times': 1154,
 'The Motley Fool': 576,
 'Yahoo Finance': 3452,
 'Financial Times News': 46}

In [16]:
#這邊要人工判斷
news_resource['Financial Times'] = news_resource['Financial Times']+news_resource['Financial Times News']
del news_resource['Financial Times News']

KeyError: 'Financial Times (FT)'

In [17]:
news_resource

{'Forbes': 908,
 'CNBC': 1655,
 'Business Insider': 937,
 'Seeking Alpha': 3572,
 'Bloomberg Business': 2383,
 'International Business Times': 267,
 'MarketWatch': 1599,
 'The Wall Street Journal': 931,
 'Financial Times': 1200,
 'The Motley Fool': 576,
 'Yahoo Finance': 3452}

In [18]:
# 建立新聞來源list
new_publishment = []
for key,value in news_resource.items():
    new_publishment.append(key)
print(new_publishment)

['Forbes', 'CNBC', 'Business Insider', 'Seeking Alpha', 'Bloomberg Business', 'International Business Times', 'MarketWatch', 'The Wall Street Journal', 'Financial Times', 'The Motley Fool', 'Yahoo Finance']


In [19]:
np.save('./result/News_resource/AUD_new_resource.npy', news_resource)

#load method
# import numpy as np
# new_dict = np.load('./result/News_resource/EUR_new_resource.npy', allow_pickle='TRUE')
# print(new_dict)

In [20]:
#把df中的重複title替換掉，其餘的若不再新聞來源list則刪除
for i in range(df_news.shape[0]):
    if df_news["source_title"][i] == 'finance.yahoo.com':
        df_news["source_title"][i] = 'Yahoo Finance'
    elif df_news["source_title"][i] == 'Yahoo! Finance':
        df_news["source_title"][i] = 'Yahoo Finance'
    elif df_news["source_title"][i] == 'Financial Times News':
        df_news["source_title"][i] = 'Financial Times'
    elif df_news["source_title"][i] == 'Financial Times (FT)':
        df_news["source_title"][i] = 'Financial Times'
    elif df_news['source_title'][i] not in new_publishment:
        df_news.drop(i,inplace=True)



In [21]:
df_news = df_news.reset_index(drop =True)

In [22]:
df_news.shape

(17480, 3)

### 添加TF-IDF欄位

In [23]:
df_news.insert(2, "tfidftext", "") 

In [30]:
df_news.insert(2, "word_segmentation", "") 

In [31]:
df_news = df_news.reset_index(drop = True)
df_news

,date,body,word_segmentation,tfidftext,source_title
0,2016-01-01,Roger Federer is a man used to making history....,,,Forbes
1,2016-01-03,We're continually told that raising the minimu...,,,Forbes
2,2016-01-04,"If you have a large tax debt, you might have t...",,,Forbes
3,2016-01-04,"As markets reopen for the first week of 2016, ...",,,CNBC
4,2016-01-04,Good morning. Here's everything you need to kn...,,,Business Insider
...,...,...,...,...,...
17475,2020-12-31,"The Los Olivos, Calif., property underwent a s...",,,The Wall Street Journal
17476,2020-12-31,A look at the biggest residential transactions...,,,The Wall Street Journal
17477,2020-12-31,I write about global policy and lead advocacy ...,,,Forbes
17478,2020-12-31,Buoyed by positive momentum for shares of Inte...,,,MarketWatch


In [25]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17480 entries, 0 to 17479
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          17480 non-null  object
 1   body          17480 non-null  object
 2   tfidftext     17480 non-null  object
 3   source_title  17480 non-null  object
dtypes: object(4)
memory usage: 546.4+ KB


In [26]:
#移除html標籤
def movehtml(text):
    dr = re.compile(r'<[^>]+>',re.S)
    return dr.sub('',text)

In [27]:
#移除文本標點以及數字
def removepunctuation(text):
    remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    return re.sub(remove_chars, '', text)

In [28]:
#去除\n\r\t
def movebackslash(text):
    text = text.replace("\r", " ")
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    return text

In [29]:
#還原縮寫字
replacement_patterns = [
(r'won\'t', 'will not'),
(r'can\'t', 'cannot'),
(r'i\'m', 'i am'),
(r'ain\'t', 'is not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)n\'t', '\g<1> not'),
(r'(\w+)\'ve', '\g<1> have'),
(r'(\w+)\'s', '\g<1> is'),
(r'(\w+)\'re', '\g<1> are'),
(r'(\w+)\'d', '\g<1> would')]

class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            (s, count) = re.subn(pattern, repl, s)
        return s
        



In [45]:
#正規化處理
replacer = RegexpReplacer()
for news_index in range(df_news.shape[0]):
    a = df_news["body"][news_index]
    b = ''.join(a) #確保a為str，而非list
    c = b.lower() #正規化，轉小寫
    result = replacer.replace(c) # 還原縮寫
    
    #斷句 ->bert用
    sentences = nltk.sent_tokenize(result)
    for i in range(len(sentences)):
        sentences[i] = removepunctuation(sentences[i]) #移除文本標點以及數字
        sentences[i] = movebackslash(sentences[i]) #去除\n\r\t
    df_news["word_segmentation"][news_index] = sentences
    
    #斷詞 -> word2vec
    result = removepunctuation(result) #移除文本標點以及數字
    result = movehtml(result) #移除html標籤
    result = movebackslash(result) #去除\n\r\t
    df_news["body"][news_index] = result
    

In [51]:
#Word Segmentation (斷詞)
for newsbody_index in range(df_news.shape[0]):
    a = df_news['body'][newsbody_index]
    newstokens = nltk.tokenize.word_tokenize(a)#將單篇文章的各句子斷詞
    df_news['tfidftext'][newsbody_index] = newstokens #將單篇文章的各句斷詞彙總到新串列，for loop 將該月所有文章集成到新串列
    

In [52]:
df_news

,date,body,word_segmentation,tfidftext,source_title
0,2016-01-01,roger federer is a man used to making history ...,[roger federer is a man used to making history...,"[roger, federer, is, a, man, used, to, making,...",Forbes
1,2016-01-03,we are continually told that raising the minim...,[we are continually told that raising the mini...,"[we, are, continually, told, that, raising, th...",Forbes
2,2016-01-04,if you have a large tax debt you might have tr...,[if you have a large tax debt you might have t...,"[if, you, have, a, large, tax, debt, you, migh...",Forbes
3,2016-01-04,as markets reopen for the first week of invest...,[as markets reopen for the first week of inves...,"[as, markets, reopen, for, the, first, week, o...",CNBC
4,2016-01-04,good morning here is everything you need to kn...,"[good morning, here is everything you need to ...","[good, morning, here, is, everything, you, nee...",Business Insider
...,...,...,...,...,...
17475,2020-12-31,the los olivos calif property underwent a seri...,[the los olivos calif property underwent a ser...,"[the, los, olivos, calif, property, underwent,...",The Wall Street Journal
17476,2020-12-31,a look at the biggest residential transactions...,[a look at the biggest residential transaction...,"[a, look, at, the, biggest, residential, trans...",The Wall Street Journal
17477,2020-12-31,i write about global policy and lead advocacy ...,[i write about global policy and lead advocacy...,"[i, write, about, global, policy, and, lead, a...",Forbes
17478,2020-12-31,buoyed by positive momentum for shares of inte...,[buoyed by positive momentum for shares of int...,"[buoyed, by, positive, momentum, for, shares, ...",MarketWatch


In [53]:
#詞性還原
for newsbody_index in range(df_news.shape[0]):
    pos = nltk.pos_tag(df_news['tfidftext'][newsbody_index])
    df_news['tfidftext'][newsbody_index] = pos

In [54]:
df_news['tfidftext'][0]

[('roger', 'NN'),
 ('federer', 'NN'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('man', 'NN'),
 ('used', 'VBN'),
 ('to', 'TO'),
 ('making', 'VBG'),
 ('history', 'NN'),
 ('but', 'CC'),
 ('when', 'WRB'),
 ('it', 'PRP'),
 ('comes', 'VBZ'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('race', 'NN'),
 ('to', 'TO'),
 ('reach', 'VB'),
 ('million', 'CD'),
 ('in', 'IN'),
 ('prize', 'JJ'),
 ('money', 'NN'),
 ('the', 'DT'),
 ('time', 'NN'),
 ('major', 'JJ'),
 ('champion', 'NN'),
 ('may', 'MD'),
 ('be', 'VB'),
 ('upstaged', 'VBN'),
 ('by', 'IN'),
 ('novak', 'JJ'),
 ('djokovic', 'JJ'),
 ('federer', 'NN'),
 ('is', 'VBZ'),
 ('oncourt', 'JJ'),
 ('earnings', 'NNS'),
 ('of', 'IN'),
 ('million', 'CD'),
 ('seemingly', 'RB'),
 ('put', 'VBD'),
 ('him', 'PRP'),
 ('in', 'IN'),
 ('pole', 'NN'),
 ('position', 'NN'),
 ('to', 'TO'),
 ('reach', 'VB'),
 ('the', 'DT'),
 ('historic', 'JJ'),
 ('mark', 'NN'),
 ('compared', 'VBN'),
 ('with', 'IN'),
 ('djokovic', 'NN'),
 ('is', 'VBZ'),
 ('million', 'CD'),
 ('however', 'RB'),
 ('the', 'DT'),
 (

In [55]:
#Lemmatization (字型還原)

for newsbody_index in range(df_news.shape[0]):
    wordnet_pos = []
    p = df_news['tfidftext'][newsbody_index]
    for word, tag in p:
        if tag.startswith('J'):
            wordnet_pos.append(nltk.corpus.wordnet.ADJ)
        elif tag.startswith('V'):
            wordnet_pos.append(nltk.corpus.wordnet.VERB)
        elif tag.startswith('N'):
            wordnet_pos.append(nltk.corpus.wordnet.NOUN)
        elif tag.startswith('R'):
            wordnet_pos.append(nltk.corpus.wordnet.ADV)
        else:
            wordnet_pos.append(nltk.corpus.wordnet.NOUN)
            
    # Lemmatizer
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    df_news['tfidftext'][newsbody_index] = [lemmatizer.lemmatize(p[n][0], pos=wordnet_pos[n]) for n in range(len(p))]
    

In [56]:
df_news['tfidftext'][0]

['roger',
 'federer',
 'be',
 'a',
 'man',
 'use',
 'to',
 'make',
 'history',
 'but',
 'when',
 'it',
 'come',
 'to',
 'the',
 'race',
 'to',
 'reach',
 'million',
 'in',
 'prize',
 'money',
 'the',
 'time',
 'major',
 'champion',
 'may',
 'be',
 'upstage',
 'by',
 'novak',
 'djokovic',
 'federer',
 'be',
 'oncourt',
 'earnings',
 'of',
 'million',
 'seemingly',
 'put',
 'him',
 'in',
 'pole',
 'position',
 'to',
 'reach',
 'the',
 'historic',
 'mark',
 'compare',
 'with',
 'djokovic',
 'be',
 'million',
 'however',
 'the',
 'topranked',
 'serb',
 'may',
 'get',
 'there',
 'first',
 'base',
 'on',
 'his',
 'outstanding',
 'performance',
 'in',
 'the',
 'past',
 'few',
 'year',
 'and',
 'a',
 'reduced',
 'schedule',
 'for',
 'the',
 'men',
 'be',
 'grand',
 'slam',
 'record',
 'holder',
 'from',
 'switzerland',
 'although',
 'the',
 'two',
 'men',
 'be',
 'tie',
 'in',
 'career',
 'meeting',
 'djokovic',
 'hand',
 'federer',
 'five',
 'defeat',
 'in',
 'eight',
 'match',
 'in',
 'inclu

In [57]:
#remove Stopword (停用詞) 
nltk_stopwords = nltk.corpus.stopwords.words("english")

for newsbody_index in range(df_news.shape[0]):
    p = df_news['tfidftext'][newsbody_index]
    tokens = [token for token in p if token not in nltk_stopwords]
    df_news['tfidftext'][newsbody_index] = tokens

#現在即可得到所有詞幹還原的單詞列表 type(tokens) 為一維list。

#查驗用
#for token in tokens:
    #print(token)

In [58]:
len(df_news['tfidftext'][12])

212

In [51]:
print(df_news.shape)

body_length = 0
tfidftext_length = 0
body_length_list = []
tfidftext_length_list = []
for i in range(df_news.shape[0]):
    
    a = df_news['body'][i]
    newstokens = nltk.tokenize.word_tokenize(a)
    body_length += len(newstokens)
    body_length_list.append(len(newstokens))
    
    tfidftext_length += len(df_news['tfidftext'][i])
    tfidftext_length_list.append(len(df_news['tfidftext'][i]))
    
print("平均文本長度為：",body_length//df_news.shape[0])    
print("去除停用詞之平均文本長度為：",tfidftext_length//df_news.shape[0])

(18489, 4)
平均文本長度為： 980
去除停用詞之平均文本長度為： 562


## 至此為止，單篇文章的處理皆完成

In [59]:
#建立一個新的dataframe來處理資料吧! -->還缺匯率漲跌的欄位後續再補。
date = []
word2vec_body = []
tfidf_body = []
bert_body = []
source_title = []

for newsbody_index in range(df_news.shape[0]):
    b = df_news['body'][newsbody_index]
    d = df_news['date'][newsbody_index]
    q = df_news['tfidftext'][newsbody_index]
    t = df_news['source_title'][newsbody_index]
    s = df_news['word_segmentation'][newsbody_index]
    word2vec_body.append(b)
    date.append(d)
    tfidf_body.append(" ".join(q))
    source_title.append(t)
    bert_body.append(s)
    
dic = {
    "date" : date,
    "word2vec_body" : word2vec_body,
    "tfidf_body" : tfidf_body,
    "bert_body" : bert_body,
    "source_title" : source_title
}

df = pd.DataFrame(dic)
df

,date,word2vec_body,tfidf_body,bert_body,source_title
0,2016-01-01,roger federer is a man used to making history ...,roger federer man use make history come race r...,[roger federer is a man used to making history...,Forbes
1,2016-01-03,we are continually told that raising the minim...,continually tell raise minimum wage hour incre...,[we are continually told that raising the mini...,Forbes
2,2016-01-04,if you have a large tax debt you might have tr...,large tax debt might trouble travel since irs ...,[if you have a large tax debt you might have t...,Forbes
3,2016-01-04,as markets reopen for the first week of invest...,market reopen first week investor focus asia c...,[as markets reopen for the first week of inves...,CNBC
4,2016-01-04,good morning here is everything you need to kn...,good morning everything need know world advert...,"[good morning, here is everything you need to ...",Business Insider
...,...,...,...,...,...
17475,2020-12-31,the los olivos calif property underwent a seri...,los olivos calif property undergo series price...,[the los olivos calif property underwent a ser...,The Wall Street Journal
17476,2020-12-31,a look at the biggest residential transactions...,look big residential transaction close around ...,[a look at the biggest residential transaction...,The Wall Street Journal
17477,2020-12-31,i write about global policy and lead advocacy ...,write global policy lead advocacy global citiz...,[i write about global policy and lead advocacy...,Forbes
17478,2020-12-31,buoyed by positive momentum for shares of inte...,buoy positive momentum share intel goldman sac...,[buoyed by positive momentum for shares of int...,MarketWatch


In [60]:
# 儲存一下資料!!!
df.to_csv("./Pre_data/news/AUDNews105109.csv")